## 라이브러리 설치

In [ ]:
#!pip install faiss-cpu ##또는 faiss-gpu
!pip install -U accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

## 기본 import 및 문서 로드&임베드

In [ ]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
# 문서가 저장된 폴더 경로
DOCUMENTS_PATH = "./doc_example"

# 폴더에서 .txt 파일 읽어오기
def load_documents_from_folder(folder_path):
    documents = []
    filenames = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # .txt 파일만 로드
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                content = file.read().strip()
                documents.append(content)
                filenames.append(filename)

    return documents, filenames

# 문서 로드
documents, filenames = load_documents_from_folder(DOCUMENTS_PATH)
# 문서 임베딩 모델 로드
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# 문서 임베딩 생성
document_embeddings = embedding_model.encode(documents)

# FAISS 인덱스 생성
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))

In [ ]:
def retrieve_relevant_documents(query, top_k=2):
    """질문에 가장 관련 있는 문서를 검색"""
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [documents[idx] for idx in indices[0]]

## GPT 모델로 답변 생성

In [ ]:
import openai

# OpenAI API 키 설정
api_key = ""
client = openai.OpenAI(api_key=api_key)

In [ ]:
def generate_answer(query):
    """검색된 문서를 바탕으로 GPT-4가 답변 생성"""
    relevant_docs = retrieve_relevant_documents(query)
    context = "\n".join(relevant_docs)

    prompt = f"""
    문맥 정보:
    {context}

    사용자의 질문:
    {query}

    위 문맥을 바탕으로 질문에 답변해줘.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "당신은 유용한 정보를 제공하는 AI입니다."},
                  {"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [ ]:
# 테스트 실행
query = input("질문을 입력하셔라: ")
answer = generate_answer(query)
print("답변:", answer)

질문을 입력하셔라: CCC에선 영적운동이 무엇이라고 말하는가
답변: CCC에서 말하는 영적운동은 지상명령성취를 위해 전도, 육성, 파송하는, 같은 마음을 가진 제자들의 팀을 통한 하나님의 일하심을 말합니다. 이는 잃어버린 영혼을 그리스도께 연결시키며 삶을 변화시키는 제자화와 승법번식하는 지도자들의 작업을 포함합니다. 주요 목표는 하나님의 말씀에 순종하여 모든 세대에서 주님의 지상명령 성취를 돕는 일에 적극적으로 참여하는 그리스도인의 운동을 일으키는 것입니다. 이렇게 계속해서 역동적으로 성장이 일어나고 삶의 변화가 일어나면서 죄의 반복이 끊기고, 부족함이 채워지며, 넘침이 덜어지는 운동을 말합니다.


## LLaMa 모델로 답변 생성(성능 낮음)

In [ ]:
import os

os.environ['HF_TOKEN']=""

In [ ]:
#!pip install transformers accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# LLaMA 3 모델명
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # 경량화된 1B Instruct 모델 사용

# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 # 4-bit 양자화 적용
    bnb_4bit_compute_dtype=torch.float16,  # FP16으로 연산
    bnb_4bit_use_double_quant=True
)

# 모델 로드 (4-bit 적용)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # GPU 자동 할당
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
# pad_token을 eos_token으로 설정
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# 모델을 올바른 장치로 이동 (예: CUDA)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# LLaMA 모델을 사용하여 답변 생성
def generate_answer_LLama3(query):
    """RAG 방식을 사용하여 답변 생성"""

    # 1. 질문에 대한 관련 문서 검색 (retrieve_relevant_documents 함수는 외부 함수로 정의되어야 합니다)
    relevant_docs = retrieve_relevant_documents(query)
    context = "\n".join(relevant_docs)

    # 2. LLaMA 모델에 대한 인풋 준비
    prompt = f"문맥 정보:\n{context}\n\n사용자의 질문: {query}\n\n답변:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)

    # 3. 입력 데이터를 GPU로 이동 (모델과 입력 데이터 모두 동일한 장치로 이동)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 4. 텍스트 생성 (모델에서 직접 텍스트 생성)
    outputs = model.generate(
        **inputs,
        num_return_sequences=1,
        max_new_tokens=200,  # 새로 생성되는 최대 토큰 수만 설정
        do_sample=True,
        top_p=0.9,
        temperature=0.5,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=False,
        )

    # 5. 생성된 텍스트 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 6. 출력에서 문맥 정보 제거 (예: "문맥 정보:" 또는 "사용자의 질문:" 이후 텍스트만 남기기)
    start_index = answer.find("답변:")  # "답변:"이 시작되는 위치 찾기
    if start_index != -1:
        answer = answer[start_index + len("답변:"):].strip()  # "답변:" 이후의 내용만 추출

    return answer

In [ ]:
# 테스트 실행
query = input("질문을 입력하셔라: ")
answer = generate_answer_LLama3(query+"\n문장 형태로 답변하라.")
print("답변:", answer)